Starter 

In [1]:
from pathlib import Path
import numpy as np 
import pandas as pd 
import plotly.express as px
import sys
import kaleido
import plotly
print("kaleido ok")


kaleido ok


In [3]:
ROOT=Path().resolve().parent
if str(ROOT/"src") not in sys.path:
    sys.path.insert(0, str(ROOT/"src"))

from data_workflow.viz import bar_sorted, time_line, histogram_chart, save_fig
from data_workflow.io import read_parquet

DATA=ROOT/ "data/processed/analytics_table.parquet"
FIGS=ROOT/"reports/figures"
FIGS.mkdir(parents=True, exist_ok=True)

print("ROOT =", ROOT)
print("DATA exists?", DATA.exists())


ROOT = D:\OneDrive\Documents\B2
DATA exists? True


In [7]:
# 1) Check user_id format in analytics table
print(df["user_id"].head().tolist(), df["user_id"].dtype)

# 2) Check country missing
print("country missing:", df["country"].isna().sum(), "/", len(df))

# 3) See joined result directly
print(df[["user_id", "country"]].head(10))


['1', '2', '3', '1', '4'] string
country missing: 5 / 5
  user_id country
0       1    None
1       2    None
2       3    None
3       1    None
4       4    None


In [8]:
orders = pd.read_parquet(ROOT / "data/processed/orders_clean.parquet")
users  = pd.read_parquet(ROOT / "data/processed/users.parquet")

print("orders:", orders["user_id"].head().tolist(), orders["user_id"].dtype)
print("users :", users["user_id"].head().tolist(), users["user_id"].dtype)

# Intersection test
print(
    "intersection:",
    len(set(orders["user_id"].astype(str)) & set(users["user_id"].astype(str)))
)


orders: ['1', '2', '3', '1', '4'] string
users : ['0001', '0002', '0003', '0004'] string
intersection: 0


Read parquet and check cleaning

In [6]:
df=pd.read_parquet(DATA)
print("rows:", len(df), "columns:" , len(df.columns))
print(df.dtypes.head(10))

missing=df.isna().sum().sort_values(ascending=False).head(10)
print(missing)

rows: 5 columns: 18
order_id               string[python]
user_id                string[python]
amount                        float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
dtype: object
signup_date    5
country        5
hour           1
day            1
amount         1
quantity       1
month          1
year           1
created_at     1
date           1
dtype: int64


Question 1: revenue by country 

In [5]:
rev=(
    df.groupby("country", dropna=False)
    .agg(  ##aggregation for analysis
        n=("order_id", "size"),
        revenue=("amount","sum"),
        aov=("amount", "mean"),
          )
        .reset_index()
        .sort_values("revenue", ascending=False)
)
fig=px.bar(rev, x="country", y="revenue", title="revenue by country")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="country")
fig.update_yaxes(title_text="revenue")
fig.write_image(str(FIGS / "revenue_by_country.png"))

fig.show()

 Question 2: Revenue trend (monthly)

In [ ]:
monthly=(
    df.groupby("month", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount","sum"),
    )
    .reset_index()
    .sort_values("month") )
fig=px.bar(monthly, x="month", y="revenue", title="revenue by month")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="month")
fig.update_yaxes(title_text="revenue")
save_fig(fig, FIGS/"revenue_by_month.png")
fig

 Question 3: Amount distribution

In [ ]:
fig=px.histogram(df,x="amount_winsor", nbins=30, title="order amount distribution")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="amount")
fig.update_yaxes(title_text="number of orders")
save_fig(fig, FIGS/"amount_of_orders.png")
fig